Veri = https://archive.ics.uci.edu/ml/datasets/AI4I+2020+Predictive+Maintenance+Dataset

In [1]:
import pandas as pd

df = pd.read_csv("ai4i2020.csv")
df.head()

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,0,0,0,0,0
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,0,0,0,0,0
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,0,0,0,0,0
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,0,0,0,0,0
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,0,0,0,0,0


Veri seti kestirimci bakım tahminlemesi üzerine çalışabilecek bir veri seti.

In [2]:
df.shape

(10000, 14)

10.000 gözlemden oluşuyor.

In [3]:
df.isnull().sum()

UDI                        0
Product ID                 0
Type                       0
Air temperature [K]        0
Process temperature [K]    0
Rotational speed [rpm]     0
Torque [Nm]                0
Tool wear [min]            0
Machine failure            0
TWF                        0
HDF                        0
PWF                        0
OSF                        0
RNF                        0
dtype: int64

Boş verimiz hiç yok. Bu iyi bir haber.

Şimdi merak ettiğimiz öznitelikleri bir inceleyelim.

In [4]:
df["Machine failure"].value_counts()

0    9661
1     339
Name: Machine failure, dtype: int64

Makine arızası bilgisi veriyor. 339 arıza kaydı bulunurken 9661 arıza olmayan örnek bulunuyor. Bu verimiz son derece çarpık olduğu anlamına geliyor. Bu bizim için bir sorun olabilir. Bununla ilgilenmemiz gerekecek.

In [5]:
df["Type"].value_counts()

L    6000
M    2997
H    1003
Name: Type, dtype: int64

Ürünlerin kalite bilgisi. One-Hot Encoding yapmamız gerekiyor.

In [6]:
#One-Hot Encoding
df_model = pd.get_dummies(df, columns = ["Type"], prefix = ["Type"])

In [7]:
df["Product ID"].value_counts()

L53706    1
H32563    1
L50490    1
L53693    1
M21232    1
         ..
L55979    1
L51243    1
M24275    1
L48488    1
M20468    1
Name: Product ID, Length: 10000, dtype: int64

In [8]:
df["UDI"].value_counts()

2047    1
5424    1
1338    1
7481    1
5432    1
       ..
2716    1
8857    1
4759    1
6806    1
2049    1
Name: UDI, Length: 10000, dtype: int64

İkisi de aynı şeyi ifade ediyor. Bizim çalışmamız için ikisine de ihtiyacımız yok. İkisini de silebiliriz.

In [9]:
df_model = df_model.drop(["Product ID", "UDI"], axis = 1)

In [10]:
df_model = df_model.drop(["TWF", "HDF", "PWF", "OSF", "RNF"], axis = 1)

Ben çalışmamda sadece arıza tahmini yapacağım. Arıza türünün tahminini yapmayacağım.

In [11]:
df_model.head()

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,Type_H,Type_L,Type_M
0,298.1,308.6,1551,42.8,0,0,0,0,1
1,298.2,308.7,1408,46.3,3,0,0,1,0
2,298.1,308.5,1498,49.4,5,0,0,1,0
3,298.2,308.6,1433,39.5,7,0,0,1,0
4,298.2,308.7,1408,40.0,9,0,0,1,0


In [12]:
df_model = df_model.rename(columns = {"Machine failure" : "Machine_failure"})

df0 = df_model[df_model.Machine_failure == 0]
print(df0.shape)

(9661, 9)


In [13]:
df2 = df0.sample(frac=0.0684,random_state=200)
df2.shape

(661, 9)

In [14]:
df1 = df_model[df_model.Machine_failure == 1]

In [15]:
df1.shape

(339, 9)

In [16]:
df_model = df2.append(df1)

In [17]:
from sklearn.utils import shuffle
df_model = shuffle(df_model).reset_index(drop=True)

In [18]:
df_model.head()

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine_failure,Type_H,Type_L,Type_M
0,298.8,310.0,1530,38.3,53,0,1,0,0
1,298.9,309.9,1590,31.3,98,0,0,1,0
2,301.8,310.3,1341,50.5,64,1,0,1,0
3,301.0,310.5,1317,50.9,75,0,0,1,0
4,300.1,309.2,1326,55.9,55,0,0,1,0


Verinin fazla çarpık olduğundan daha önce bahsetmiştim. Makine arızası sütununun çoğunluğu 0'dı. Bu çarpıklığı engellemek için toplam veri satırını 1000'e tamamlayacak şekilde makine arızası değeri 0 olanlar içinde rastgele seçim yaptık ve bunların içinden 661 örneği aldık sadece. Bu örneklerle 339 satırlık arızalı sütunları birleştirince 1000 satırlık, çarpık olmayan yeni bir veri setimiz oldu. Modeli bu veri seti üzerinden kuracağız.

In [19]:
df_model.corr()

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine_failure,Type_H,Type_L,Type_M
Air temperature [K],1.000000,0.859390,-0.084544,0.078978,-0.013182,0.194995,-0.063584,-0.014387,0.057839
Process temperature [K],0.859390,1.000000,-0.032765,0.025699,0.004277,0.076533,-0.059802,-0.012933,0.053745
Rotational speed [rpm],-0.084544,-0.032765,1.000000,-0.833030,-0.059672,-0.071157,-0.017717,-0.033425,0.048185
Torque [Nm],0.078978,0.025699,-0.833030,1.000000,0.078603,0.382815,-0.013537,0.044130,-0.039136
Tool wear [min],-0.013182,0.004277,-0.059672,0.078603,1.000000,0.259807,0.012465,0.030611,-0.041635
Machine_failure,0.194995,0.076533,-0.071157,0.382815,0.259807,1.000000,-0.076570,0.067799,-0.023151
Type_H,-0.063584,-0.059802,-0.017717,-0.013537,0.012465,-0.076570,1.000000,-0.434464,-0.189312
Type_L,-0.014387,-0.012933,-0.033425,0.044130,0.030611,0.067799,-0.434464,1.000000,-0.802153
Type_M,0.057839,0.053745,0.048185,-0.039136,-0.041635,-0.023151,-0.189312,-0.802153,1.000000


In [20]:
df_model.corr().Machine_failure

Air temperature [K]        0.194995
Process temperature [K]    0.076533
Rotational speed [rpm]    -0.071157
Torque [Nm]                0.382815
Tool wear [min]            0.259807
Machine_failure            1.000000
Type_H                    -0.076570
Type_L                     0.067799
Type_M                    -0.023151
Name: Machine_failure, dtype: float64

Korelasyon değerleri hesaplandığında makine arızasını etkileyen en önemli değişkenin tork olduğunu ve diğer değişkenlerinde ne önemde etkilediğini gözlemleyebiliyoruz.

In [21]:
X = df_model.drop("Machine_failure", axis = 1)
Y = df_model.Machine_failure

In [22]:
from sklearn import preprocessing

#Standard Scaler uyguluyoruz bütün sütunlara
scaler = preprocessing.StandardScaler()

scaler.fit(X)
X_model = scaler.transform(X)

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_model,Y,test_size=0.3,random_state=0)

### Karar Ağacı

In [24]:
from sklearn import tree

In [25]:
karar_agaci = tree.DecisionTreeClassifier()

In [26]:
karar_agaci.fit(X_train, y_train)

DecisionTreeClassifier()

In [27]:
from sklearn import metrics

metrics.accuracy_score(y_test, karar_agaci.predict(X_test))

0.91

### Lojistik Regresyon

In [28]:
from sklearn.linear_model import LogisticRegression

loj_model = LogisticRegression(solver = "liblinear").fit(X_train, y_train)

In [29]:
metrics.accuracy_score(y_test, loj_model.predict(X_test))

0.8133333333333334

### KNN

In [30]:
from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier().fit(X_train, y_train)

In [31]:
metrics.accuracy_score(y_test, knn_model.predict(X_test))

0.88

### SVM

In [32]:
from sklearn.svm import SVC

svm_model = SVC(kernel = "linear").fit(X_train, y_train)

In [33]:
metrics.accuracy_score(y_test, svm_model.predict(X_test))

0.7966666666666666

### Yapay Sinir Ağları

In [34]:
from sklearn.neural_network import MLPClassifier

mlpc_model = MLPClassifier().fit(X_train, y_train)

D:\Program Files (x86)\Anaconda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [35]:
metrics.accuracy_score(y_test, mlpc_model.predict(X_test))

0.8833333333333333

### Random Forest

In [36]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier().fit(X_train, y_train)

metrics.accuracy_score(y_test, rf_model.predict(X_test))

0.93

Kullanılan sınıflandırma algoritmaları arasında en başarılı sonucu %93 ile Random Forest algoritması ile kurduğumuz model verdi. Kendisini üstün başarısından ötürü tebrik ediyoruz.